In [24]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers.legacy import Adam

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)
import random

### Data Loading

In [2]:
out_dir = Path("../phys-vae/data/climate/")
data = pd.read_parquet("complete_data.parquet")

### Cleaning and NaN removal

In [3]:
# Replace -9999 with NaN
data['NEE'] = data['NEE'].replace(-9999, np.nan)

# Identify the consecutive gaps
data['Gap'] = data['NEE'].isna().astype(int).groupby(data['NEE'].notna().astype(int).cumsum()).cumsum()

# Filter out the zero values and calculate the lengths of the gaps
gap_lengths = data[data['Gap'] != 0]['Gap'].value_counts().reset_index()
gap_lengths.columns = ['Gap Length', 'Frequency']

gap_lengths

,Gap Length,Frequency
0,1,8619
1,2,4881
2,3,3767
3,4,3013
4,5,2540
...,...,...
810,751,1
811,749,1
812,748,1
813,747,1


In [4]:
data = data.dropna().reset_index(drop=True)

### Adding features

In [5]:
def set_stats(df, flux, scale="30T"):
    df = df.copy()
    ind = df.index
    dt = df["DateTime"]
    df.set_index('DateTime', inplace=True)
    flux_orig = df[flux].copy()
    df[flux] = df[flux].interpolate().values
    
    # Set max
    flux_max = df[flux].resample("D").max()
    df["flux_max"] = flux_max.resample(scale).bfill()
    
    # Set min
    flux_min = df[flux].resample("D").min()
    df["flux_min"] = flux_min.resample(scale).bfill()
    
    # Set mean
    flux_mean = df[flux].resample("D").mean()
    df["flux_mean"] = flux_mean.resample(scale).bfill()
    
    # Set std
    flux_std = df[flux].resample("D").std()
    df["flux_std"] = flux_std.resample(scale).bfill()
    
    # Set 25%, 50%, 75% quantiles
    flux_p25 = df[flux].resample("D").quantile(0.25)
    df["flux_p25"] = flux_p25.resample(scale).bfill()
    
    flux_p50 = df[flux].resample("D").quantile(0.50)
    df["flux_p50"] = flux_p50.resample(scale).bfill()
    
    flux_p75 = df[flux].resample("D").quantile(0.75)
    df["flux_p75"] = flux_p75.resample(scale).bfill()
    
    df = df.interpolate()
    
    df.index = ind
    df["DateTime"] = dt.values
    df.loc[:, flux] = flux_orig.values
    return df, ["flux_max", "flux_min", "flux_mean", "flux_std", "flux_p25", "flux_p50", "flux_p75"]
 
#====================================================================================================
# set season tag
def set_season_tag(df, isnorth = True):
    if isnorth:
        df["season"] = (df['DateTime'].month%12 + 3) // 3 # print(seasons)
    else:
        df["season"] = ((df['DateTime'].month + 6)%12 + 3)//3
    return df, ["season"]
#====================================================================================================
# set radiance tag

def set_rg_tag(df, rg):
    df["rg_rank"] = np.select(
        condlist = [
            df[rg] < 10,
            (df[rg] > 10) & (df[rg] < 100),
            df[rg] > 100
        ],
        choicelist = [
            1,
            2,
            3
        ],
        default = 0
    )
    return df, ["rg_rank"]

def extract_features(data):
    """
    Extract features from the timestamp for model training.
    
    Parameters:
    data (pd.DataFrame): DataFrame with 'Timestamp' and 'NEE' columns.
    
    Returns:
    pd.DataFrame: DataFrame with extracted features.
    """
    data = data.copy()
    data['hour'] = data['DateTime'].dt.hour
    data['dayofweek'] = data['DateTime'].dt.dayofweek
    data['month'] = data['DateTime'].dt.month
    data['dayofyear'] = data['DateTime'].dt.dayofyear
    return data

In [6]:
data, stat_tags = set_stats(data, "NEE")
data, rg_tag = set_rg_tag(data, 'Rg')

In [7]:
data, stat_tags = set_stats(data, "NEE")
data, rg_tag = set_rg_tag(data, 'Rg')
data = extract_features(data)

In [8]:
data.head(10)

,NEE,NEE_unc,LE,LE_unc,H,H_unc,Tau,Tau_unc,CO2_strg,LE_strg,...,flux_std,flux_p25,flux_p50,flux_p75,rg_rank,DateTime,hour,dayofweek,month,dayofyear
0,12.5670,3.06210,-9999.000,14.6310,-2.9117,0.72031,0.17393,0.012095,-0.103920,-0.512280,...,0.986102,4.82125,5.3522,5.86985,1,2012-06-22 00:30:00,0,4,6,174
1,6.7892,2.06360,37.362,12.3280,-10.5920,0.64084,0.17662,0.012124,0.089560,-0.189890,...,0.986102,4.82125,5.3522,5.86985,1,2012-06-22 01:00:00,1,4,6,174
2,4.0302,0.54104,35.169,4.7136,-15.9840,1.20660,0.18260,0.013019,-0.204920,-0.404700,...,0.986102,4.82125,5.3522,5.86985,1,2012-06-22 01:30:00,1,4,6,174
3,5.0909,0.33387,32.364,3.5567,-13.9460,0.63822,0.12357,0.008703,-0.035944,-0.124000,...,0.986102,4.82125,5.3522,5.86985,1,2012-06-22 02:00:00,2,4,6,174
4,5.8500,0.44881,28.569,3.8587,-14.0660,0.77249,0.15957,0.011172,0.074542,0.005818,...,0.986102,4.82125,5.3522,5.86985,1,2012-06-22 02:30:00,2,4,6,174
5,4.1455,0.98754,24.648,12.6020,-14.1820,0.76101,0.13249,0.009131,0.025845,0.143340,...,0.986102,4.82125,5.3522,5.86985,1,2012-06-22 03:00:00,3,4,6,174
6,3.4408,0.80687,11.048,8.1544,-14.9320,0.80979,0.13693,0.009080,0.201620,0.302600,...,0.986102,4.82125,5.3522,5.86985,1,2012-06-22 03:30:00,3,4,6,174
7,3.4772,0.80788,36.742,8.4677,-15.0720,1.20000,0.13514,0.008611,-0.022391,0.281180,...,0.986102,4.82125,5.3522,5.86985,2,2012-06-22 04:00:00,4,4,6,174
8,5.4707,0.66564,13.321,5.3211,-12.6100,0.75674,0.14830,0.009997,0.147020,0.222110,...,0.986102,4.82125,5.3522,5.86985,2,2012-06-22 04:30:00,4,4,6,174
9,4.8830,0.53958,14.134,3.1002,-11.6310,1.00730,0.11883,0.007229,-0.068287,0.151030,...,0.986102,4.82125,5.3522,5.86985,2,2012-06-22 05:00:00,5,4,6,174


### Sequence Data

In [9]:
# Features for training
drivers = ['LE', 'H', 'Tau', 'LE_strg', 'Ta', 'RH', 'VPD', 'Rg', 'Ustar', 'Tsoil1', 'Tsoil2']

#-------------------------------------------------
# prepare and split data for regressor
columns_to_pick = drivers + stat_tags + rg_tag + ['hour', 'dayofweek', 'month', 'dayofyear']

In [10]:
def split_data(data, test_size=0.2):
    """
    Split data into training and test sets.
    
    Parameters:
    data (pd.DataFrame): DataFrame with 'Timestamp' and 'NEE' columns.
    test_size (float): Proportion of the data to include in the test split.
    
    Returns:
    tuple: Training and test DataFrames.
    """
    train_data, test_data = train_test_split(data, test_size=test_size, shuffle=False)
    return train_data.reset_index(drop=True), test_data.reset_index(drop=True)

In [11]:
train_data, test_data = split_data(data)

In [12]:
X_train = train_data[columns_to_pick]
X_train = pd.get_dummies(X_train, columns=['hour', 'dayofweek', 'month']).values
Y_train = train_data["NEE"].values

X_test = test_data[columns_to_pick]
X_test = pd.get_dummies(X_test, columns=['hour', 'dayofweek', 'month']).values
Y_test = test_data["NEE"].values

In [13]:
ss = StandardScaler()
ss.fit(X_train)
X_train = ss.transform(X_train)
X_test = ss.transform(X_test)

In [27]:
def build_model(input_shape):
    model = Sequential()
    model.add(Dense(32, input_shape=(input_shape,), activation="relu", kernel_initializer="he_normal"))
    model.add(Dense(16, activation="relu", kernel_initializer="he_normal"))
    model.add(Dense(8, activation="relu", kernel_initializer="he_normal"))
    model.add(Dense(1, kernel_initializer="he_normal"))
    model.compile(optimizer=Adam(), loss="mse")
    return model

In [28]:
model = build_model(X_train.shape[1])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 32)                2048      
                                                                 
 dense_10 (Dense)            (None, 16)                528       
                                                                 
 dense_11 (Dense)            (None, 8)                 136       
                                                                 
 dense_12 (Dense)            (None, 1)                 9         
                                                                 
Total params: 2721 (10.63 KB)
Trainable params: 2721 (10.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(X_train, Y_train.reshape((-1, 1)), epochs=2000, batch_size=None, validation_data=(X_test, Y_test))

Epoch 1/2000
1552/1552 [==============================] - 1s 300us/step - loss: 12.9340 - val_loss: 11.6164
Epoch 2/2000
1552/1552 [==============================] - 0s 272us/step - loss: 9.4410 - val_loss: 10.9634
Epoch 3/2000
1552/1552 [==============================] - 0s 269us/step - loss: 8.9191 - val_loss: 10.7968
Epoch 4/2000
1552/1552 [==============================] - 0s 269us/step - loss: 8.4694 - val_loss: 11.0584
Epoch 5/2000
1552/1552 [==============================] - 0s 268us/step - loss: 8.1502 - val_loss: 11.2018
Epoch 6/2000
1552/1552 [==============================] - 0s 268us/step - loss: 7.9371 - val_loss: 11.4305
Epoch 7/2000
1552/1552 [==============================] - 0s 268us/step - loss: 7.8099 - val_loss: 10.9845
Epoch 8/2000
1552/1552 [==============================] - 0s 268us/step - loss: 7.6897 - val_loss: 11.2821
Epoch 9/2000
1552/1552 [==============================] - 0s 268us/step - loss: 7.5527 - val_loss: 11.5059
Epoch 10/2000
1552/1552 [===========

In [26]:
preds = model.predict(X_test)
print(mean_squared_error(Y_test, preds), r2_score(Y_test, preds))

388/388 [==============================] - 0s 169us/step
13.238028350384157 0.6457969516630517


In [20]:
from sklearn.ensemble import RandomForestRegressor


# Train Random Forest model
rfr = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rfr.fit(X_train, Y_train)

RandomForestRegressor(n_jobs=-1, random_state=42)

In [25]:
preds = rfr.predict(X_test)
print(mean_squared_error(Y_test, preds), r2_score(Y_test, preds))

10.690683845885678 0.7139549254017582


In [11]:
def prepare_data(data, gap_type):

    time_deltas = {
        "very short": pd.Timedelta(minutes=30),
        "short": pd.Timedelta(hours=4),
        "medium": pd.Timedelta(hours=32),
        "week": pd.Timedelta(days=7),
        "long": pd.Timedelta(days=12),
        "month": pd.Timedelta(days=30)
    }
    
    gap_length = time_deltas[gap_type]
    
    sequences = []
    targets = []
    
    for i in tqdm(range(len(data[:5000]))):
        seq_end_time = data['DateTime'].iloc[i + sequence_length - 1]
        target_end_time = seq_end_time + gap_length
        
        if target_end_time > data['DateTime'].iloc[-1]:
            break
        
        seq = data.iloc[i:i + sequence_length]
        target = data.loc[(data['DateTime'] > seq_end_time) & (data['DateTime'] <= target_end_time), 'NEE'].values
        
        if len(target) > 0 and not np.isnan(target).any():
            sequences.append(seq)
            targets.append(target)
    
    X = np.array([seq[columns_to_pick].values for seq in sequences])
    y = targets  # Keeping the targets as a list to handle dynamic lengths
    
    return X, y

# Example Usage for Version 2
X, Y = prepare_lstm_data_version2(data, sequence_type="short", gap_type="short")


100%|█████████████████████████████████████████████████████████| 4992/4992 [00:01<00:00, 2682.62it/s]


In [12]:
with open("lstm_data.pickle", "wb") as fp:
    pickle.dump({"X": X, "Y": Y}, fp)